In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client=OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
import os

In [ ]:
response = client.chat.completions.create(model='gpt-4o-mini', messages=[{"role": "user", "content": "is it to late to join the course?"}])